In [349]:
import requests
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd 

In [414]:
def embajadas():
    response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
    results = response.json()
    embajadas = pd.json_normalize(results['@graph'])
    embajadas["Dirección"] = embajadas["address.street-address"]
    return embajadas.to_json("../data/embajadas.json", orient='records')
def estaciones():
    estaciones = pd.read_json("../data/bicimad2.json", orient='records')
    geometry_coordinates = estaciones["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    estaciones = pd.concat([stations, geometry_coordinates], axis=1)
    return estaciones.to_json("../data/estaciones.json", orient='records')


In [415]:
embajadas = pd.json_normalize(results['@graph'])
embajadas

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7933,Consulado de Argentina,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28010,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,"Metro: Alonso Martínez (líneas 4, 5 y 10). ...",3,,,Consulado de Argentina
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197947,Consulado de Belice,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28016,CALLE TALAVERA 9,40.453376,-3.680978,"Metro: Concha Espina (línea 9), Colombia (lí...",3,,,Consulado de Belice
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,21112,Consulado de Bolivia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28002,CALLE AVIADOR LINDBERGH 3,40.442447,-3.679665,"Metro: Cruz del Rayo (línea 9), Prosperidad ...",3,,,Consulado de Bolivia
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,64260,Consulado de Brasil,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28001,CALLE GOYA 5 y 7 pasaje,40.425716,-3.688868,"Metro: Serrano (línea 4), Colón (línea 4). ...",3,,,Consulado de Brasil
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10197938,Consulado de Bélgica,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28046,PASEO CASTELLANA 18 PLANTA 6 Izquierda,40.428626,-3.689004,"Metro: Serrano (línea 4), Colón (línea 4), R...",3,,,Consulado de Bélgica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4650495,Embajada de Vietnam,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28016,AVENIDA ALFONSO XIII 54,40.452934,-3.667175,Metro: Avenida de la Paz (línea 4). Bus: 40...,3,,,Embajada de Vietnam
158,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,99108,Embajada de Yemen,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28046,PASEO CASTELLANA 117 PLANTA 8 D,40.457056,-3.690768,"Metro: Cuzco (línea 10). Bus: 147, 27, 40, ...",3,,,Embajada de Yemen
159,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7646811,Embajada de la República Popular Democrática d...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28023,CALLE DARIO APARICIO 43,40.472000,-3.770879,Bus: 161.,3,,,Embajada de la República Popular Democrática d...
160,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7866,Embajada de la República de Corea,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...

In [416]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)
    


In [417]:
embajadas = pd.read_json('../data/embajadas.json')
estaciones = pd.read_json('../data/estaciones.json')

In [418]:
response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
results = response.json()
json_normalize(results(['@graph']))

NameError: name 'json_normalize' is not defined

In [421]:
embajadas["mercator_start"] = embajadas.apply(lambda x: to_mercator(x["LATITUD"], x["LONGITUD"]), axis=1)
estaciones["mercator_finish"] = estaciones.apply(lambda x: to_mercator(x["LATITUD"], x["LONGITUD"]), axis=1)


In [422]:
estaciones_clean = estaciones[["name","address","LONGITUD","LATITUD", "mercator_finish"]]
estaciones_clean = estaciones_clean.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Longitud_final", "LATITUD": "Latitud_final"})


In [423]:
embajadas_clean = embajadas[["NOMBRE","Dirección","LONGITUD","LATITUD", "mercator_start"]]
embajadas_clean = embajadas_clean.rename(columns={"NOMBRE": "Place of interest", "Dirección": "Place address", "LONGITUD": "Longitud_inicial", "LATITUD": "Latitud_inicial"}, errors="raise")


In [424]:
df_resultado = pd.merge(embajadas_clean,estaciones_clean, how='cross')

In [425]:
prueba = df_resultado.head()
prueba

,Place of interest,Place address,Longitud_inicial,Latitud_inicial,mercator_start,BiciMAD station,Station location,Longitud_final,Latitud_final,mercator_finish
0,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837)
1,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782)
2,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317)
3,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148262)
4,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012)


In [426]:
prueba["Distancia"] = prueba.apply(lambda x: distance_meters(x["mercator_start"], x["mercator_finish"]), axis=1)
prueba


/Users/Andrea/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Place of interest,Place address,Longitud_inicial,Latitud_inicial,mercator_start,BiciMAD station,Station location,Longitud_final,Latitud_final,mercator_finish,Distancia
0,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837),1595.648926
1,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782),1570.712528
2,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317),1697.402416
3,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148262),1606.478855
4,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012),1107.400364
